In [1]:
from pyquil.quil import Program
from pyquil.api import QVMConnection
from pyquil.gates import *
from pyquil.api import CompilerConnection
import numpy as np
from math import pi

In [2]:
qvm = QVMConnection()

A simple demo of the Quantum Virtual Machine is to print out the action of 
CPHASE gate on states |00>, |01>, |10>, |11>

In [3]:
p = Program()
p.inst(CPHASE(-pi/16.0, 0, 1))
wf = qvm.wavefunction(p)
print(wf)

p = Program()
p.inst(X(0))
p.inst(CPHASE(-pi/16.0, 0, 1))
wf = qvm.wavefunction(p)
print(wf)

p = Program()
p.inst(X(1))
p.inst(CPHASE(-pi/16.0, 0, 1))
wf = qvm.wavefunction(p)
print(wf)

p = Program()
p.inst(X(0))
p.inst(X(1))
p.inst(CPHASE(-pi/16.0, 0, 1))
wf = qvm.wavefunction(p)
print(wf)

(1+0j)|00>
(1+0j)|01>
(1+0j)|10>
(0.9807852804-0.195090322j)|11>


A slightly more complicated task is to build exp(iJZZ) gate:

In [4]:
p = Program()
p.inst(RZ(pi/16.0, 0))
p.inst(RZ(pi/16.0, 1))
p.inst(CPHASE(-pi/8.0, 0, 1))
wf = qvm.wavefunction(p)
print(wf)

p = Program()
p.inst(X(0))
p.inst(RZ(pi/16.0, 0))
p.inst(RZ(pi/16.0, 1))
p.inst(CPHASE(-pi/8.0, 0, 1))
wf = qvm.wavefunction(p)
print(wf)

p = Program()
p.inst(X(1))
p.inst(RZ(pi/16.0, 0))
p.inst(RZ(pi/16.0, 1))
p.inst(CPHASE(-pi/8.0, 0, 1))
wf = qvm.wavefunction(p)
print(wf)

p = Program()
p.inst(X(0))
p.inst(X(1))
p.inst(RZ(pi/16.0, 0))
p.inst(RZ(pi/16.0, 1))
p.inst(CPHASE(-pi/8.0, 0, 1))
wf = qvm.wavefunction(p)
print(wf)

(0.9807852804-0.195090322j)|00>
(1+0j)|01>
(1+0j)|10>
(0.9807852804-0.195090322j)|11>


Normally the noise relaxes the qubits to the |0> state. It can be seen by running the identity circuit. On ideal simulator, the effect is not seen - the final state is the same as the initial state:

In [5]:
num_flips = 5

p = Program()
for i in range(300):
    p.inst(I(0))
p.measure(0, 0)
print(qvm.run(p, [0], num_flips))

p = Program()
p.inst(X(0))
for i in range(300):
    p.inst(I(0))
p.measure(0, 0)
qvm.run(p, [0], num_flips)

[[0], [0], [0], [0], [0]]


[[1], [1], [1], [1], [1]]

In a noise simulator, we get maximally mixed state: 50% chance of |0> and 50% chance of |1>:

In [6]:
gate_noise_probs = [0.1, 0.1, 0.1]
meas_noise_probs = [0.1, 0.1, 0.1]
noisy_qvm = QVMConnection(gate_noise=gate_noise_probs, measurement_noise=meas_noise_probs)

In [8]:
num_flips = 1000

p = Program()
for i in range(300):
    p.inst(I(0))
p.measure(0, 0)
results =noisy_qvm.run(p, [0], num_flips)
print(results[0:5])
print(np.sum(results)/num_flips)

p = Program()
p.inst(X(0))
for i in range(300):
    p.inst(I(0))
p.measure(0, 0)
results =noisy_qvm.run(p, [0], num_flips)
print(results[0:5])
print(np.sum(results)/num_flips)

[[0], [0], [0], [1], [1]]
0.494
[[0], [1], [1], [1], [1]]
0.499


On a real device, |0> is the preferred state:

In [9]:
from pyquil.api import QPUConnection
from pyquil.quil import Pragma
qpu = QPUConnection('19Q-Acorn')

In [16]:
num_flips = 1000

p = Program()
p.inst(Pragma("PRESERVE_BLOCK"))
for i in range(599):
    p.inst(I(0))
p.inst(Pragma("END_BLOCK"))
p.measure(0, 0)
results =qpu.run(p, [0], num_flips)
print(results[0:5])
print(np.sum(results)/num_flips)

p = Program()
p.inst(Pragma("PRESERVE_BLOCK"))
p.inst(X(0))
for i in range(599): #599 is the maximum number of gates allowed here
    p.inst(I(0))
p.inst(Pragma("END_BLOCK"))
p.measure(0, 0)
results =qpu.run(p, [0], num_flips)
print(results[0:5])
print(np.sum(results)/num_flips)

job jdhFcJOhTXvuIOEd is currently queued for compilation
[[0], [0], [0], [0], [0]]
0.046
job LJUMUjJJcsNimSab is currently compiling
[[0], [0], [0], [0], [0]]
0.256


Here we also added compiler directives not to simplify the circuit just in case.

The first idea is to investigate how much can this relaxation be altered by repeating pulse sequences on the qubit a-la Dynamical Decoupling. The said Dynamical decoupling usually does not alter the end state of relaxation (steady state), but extends the qubit's lifetime. Unfortunately, at the moment it's not clear how to access the wall clock in the code, so it's hard to characterize the boost in the relaxation time. Indeed, gate I takes one amount of time while gates X,Z take a different amount of time, and we don't know how to compare the results between two relaxation time measurements.

What we do instead is try to alter the steady state of the relaxation with the decoupling sequences. A slow-noise inspired sequence X Phase(omega +h) X Phase(omega-h) should make the qubit steady state point along h for the appropriate choice of omega depending on the qubit frequency. That doesn't work for any omega:

In [18]:
num_flips = 1000
angl=pi/8.0
h0=2.0
omegat=-1.0

p = Program()
p.inst(Pragma("PRESERVE_BLOCK"))
for i in range(299): #Here half of the above is the allowed # of gates
    p.inst(RZ((omegat+h0)*angl, 0))    
    p.inst(X(0))
    p.inst(RZ((omegat-h0)*angl, 0))
    p.inst(X(0))
p.inst(Pragma("END_BLOCK"))
p.measure(0, 0)
results =qpu.run(p, [0], num_flips)
print(results[0:5])
print(np.sum(results)/num_flips)

p = Program()
p.inst(Pragma("PRESERVE_BLOCK"))
p.inst(X(0))
for i in range(299):
    p.inst(RZ((omegat+h0)*angl, 0))    
    p.inst(X(0))
    p.inst(RZ((omegat-h0)*angl, 0))
    p.inst(X(0))
p.inst(Pragma("END_BLOCK"))
p.measure(0, 0)
results =qpu.run(p, [0], num_flips)
print(results[0:5])
print(np.sum(results)/num_flips)

job XbnCWrjbEAkIaYuh is currently compiling
job XbnCWrjbEAkIaYuh is currently running
[[1], [0], [1], [0], [0]]
0.572
job oducowGnvlUBCscw is currently compiling
job oducowGnvlUBCscw is currently running
[[1], [0], [0], [0], [1]]
0.532


The data: 
#|1> omega h
#480 6 0
#491 6 0
#459,474,457 0 0
#463,472 -6 0
#468 -1 0
#490 -2 0
#470 -3 0
#500 -4 0
#465 -5 0
#484 -7 0
#525 0 1
#580,482,485,504,479 0 2
#502 0 3
#512 0 4
#540 0 5
#477 0 6
#414,517,489 0 7
#477 0 -1
#490 0 -2
#489 1 2
#469 2 2
#372,485,473 3 2
#451,463 4 2
#502 -4 2
#483 -1 2
However, a simpler approach works. If we can wait in between X, then the sequence X-wait-X ends with a high (0.76) probability of |1>.

In [23]:
num_flips = 1000

p = Program()
p.inst(Pragma("PRESERVE_BLOCK"))
for i in range(39):    
    p.inst(X(0))
    p.inst(I(0))
    p.inst(I(0))
    p.inst(I(0))
    p.inst(I(0))
    p.inst(I(0))
    p.inst(I(0))
    p.inst(I(0))
    p.inst(I(0))
    p.inst(I(0))
    p.inst(I(0))
    p.inst(I(0))
    p.inst(I(0))
    p.inst(I(0))
    p.inst(X(0))
p.inst(Pragma("END_BLOCK"))
p.measure(0, 0)
results =qpu.run(p, [0], num_flips)
print(results[0:5])
print(np.sum(results)/num_flips)

p = Program()
p.inst(Pragma("PRESERVE_BLOCK"))
p.inst(X(0))
for i in range(39):
    p.inst(X(0))
    p.inst(I(0))
    p.inst(I(0))
    p.inst(I(0))
    p.inst(I(0))
    p.inst(I(0))
    p.inst(I(0))
    p.inst(I(0))
    p.inst(I(0))
    p.inst(I(0))
    p.inst(I(0))
    p.inst(I(0))
    p.inst(I(0))
    p.inst(I(0))
    p.inst(X(0))
p.inst(Pragma("END_BLOCK"))
p.measure(0, 0)
results =qpu.run(p, [0], num_flips)
print(results[0:5])
print(np.sum(results)/num_flips)

job VWnfxnGOefjmGCrl is currently queued at position 0. Estimated time until execution: 0.0 seconds.
[[1], [1], [1], [1], [1]]
0.728
job VjHjWgNToWtOheJX is currently queued for compilation
[[1], [1], [1], [1], [1]]
0.81


The sequence X Phase(omega +h) X Phase(omega-h) was attempting to unrotate e^{iHt} of the physical qubit with the omega term. The most trivial way to unrotate it is to apply Phase(omega) gates by themselves. Unfortunately, they are virtual, therefore no pulses are actually sent to the system. There is no reason to expect the environment to see our system any differently. The hope was that the next level sequence will do something interesting to cancel the e^{iHt}. We also tried it for 2 qubits with the e^{iJZZ} as the target evolution after cancelling e^{iHt}. Neither the slow-noise inspired method nor the wait-inspired methods produced relaxation to the ground state of JZZ. Below I show the code for the wait-inspired method:

We produce the maximally mixed state of qubit 0, while allowing qubit 5 to relax to |0>. We then apply CNOT for a short time between the two. That should produce a correlated state p(|00>) =0.5, p(|11>)=0.5. Instead the steady state seems to be the mixed state of two qubits. We attribute it to gate error overwhelming the thermal error.

In [24]:

p = Program()
p.inst(Pragma("PRESERVE_BLOCK"))
p.inst(X(0))
p.inst(X(5))
for i in range(15):   
    p.inst(H(5))
    p.inst(CZ( 0, 5))
    p.inst(H(5))
    p.inst(H(5))
    p.inst(CZ( 0, 5))
    p.inst(H(5))
    for i in range(30):
        p.inst(X(0))


p.inst(H(5))
p.inst(CZ( 0, 5))
p.inst(H(5))
p.inst(Pragma("END_BLOCK"))
p.measure(0, 0).measure(5, 1)

utcomes =qpu.run(p, [0,1], 1000)
counterr=[0,0,0,0]
for x in utcomes:
    if x[0]==0:
        if x[1]==0:
            counterr[0]=counterr[0]+1
        else:
            counterr[1]=counterr[1]+1
    else:
        if x[1]==0:
            counterr[2]=counterr[2]+1
        else:
            counterr[3]=counterr[3]+1
print(counterr)

job KUZBbMLxaBwvNUaB is currently compiling
[251, 366, 159, 224]


Results are:
[251, 366, 159, 224]
[233, 247, 256, 264]